## Root finding

Chapter 4. https://tobydriscoll.net/fnc-julia/nonlineqn/overview.html

We are trying to solve $f(x) = 0$ where $f$ is written in computer code. The numbers $x$ and $f(x$) are represented in floating point.

We want to write a "general" algorithm that doesn't need to know anything about the computer code. One exception will be a tool called "automatic differentiation" which will come in handy and will be explained as part of Chapter 5.

## Condition number

Write $\tilde x = x + h$ for the floating point value of $x$. If $x$ is a root, then we can usually only evaluate $f(\tilde x) = \epsilon$ where $|\epsilon|$ is small but in general not exactly 0.

Use Taylor's theorem to estimate a relationship between $h$ and $\epsilon$:

$f(x+h) = \epsilon$ so $f(x) + hf'(x) \approx \epsilon$. 

The condition number of the root finding problem is the relative error in the root ($x$) divided by the relative error in $f(x)$. That's $h/\epsilon$. Using $f(x) = 0$ and rearranging we obtain $\kappa = | \frac{h}{\epsilon} | =  |\frac{1}{f'(x)}|$. If the derivative of $f$ is 0 at the root, the condition number is $\infty$.

This means, for example, that functions with double roots may be challenging as the derivative will be zero at the root. Think of $(x-1)^2$.

We will often use $f(\tilde x)$, or $f$ evaluated at our candidate root to approximate the error. This is sometimes called the "residual" or "backward error" (not the error in x, but the error in $f(x)$). It has the benefit of being easy to evaluate!

## Newton's method

We started with a binary search, then used a secant line search. Here we use the derivative instead of the secant slope. 

Starting at a value $x_0$ we write $f(x_1)$ where $x_1 = x_0 + h$ using a tangent line approximation:

$$f(x_1) = f(x_0) + hf'(x_0).$$

We want to find an $x_1$ that is close to a root, so we set $f(x_1)=0$ and write $h = -f(x_0)/f'(x_0)$. Since $h$ is the distance between the two guesses we have a formula for our next guess:
    
$$x_1= x_0 - \frac{f(x_0)}{f'(x_0)}.$$

Clearly we need to know $f'$ and it better not be $0$ anywhere we evaluate it.

## Write your own Newton's method.

Follow the steps in class: Think. English. Julia. Small steps. Test.

Input data: $f$, $f'$, $x_0$.

Stop when $f(x)$ is close to 0, or $x_{n+1}-x_n$ is close to 0, or we've taken "too many" steps.

In [1]:
function my_newton(f, df, x0)
    tolerance = 1e-15
    x1 = 0.0
    for i in 1:20
        x1 = x0 - f(x0)/df(x0)
        if (abs(x1-x0) < tolerance) || (abs(f(x1)) < tolerance)
            break
        end
        x0 = x1
    end
    x1
end

my_newton (generic function with 1 method)

In [2]:
my_newton(x -> x - 2.0, x -> 1, 1.0)

2.0

In [3]:
my_newton(x -> x^2 - 2.0, x -> 2*x, 1.0)

1.4142135623730951

In [4]:
sqrt(2)

1.4142135623730951

In [17]:
my_newton(x -> x^3 - 2.0, x -> 3*x^2, 1.0)

1.2599210498948732

In [18]:
2.0^(1/3)

1.2599210498948732

Modify the code to keep the full sequence $x_i$.

In [5]:
function my_newton2(f, df, x0; tolerance = 1e-15)
    sequence = [ x0 ] 
    for i in 1:20
        x1 = x0 - f(x0)/df(x0)
        push!(sequence, x1)
        if (abs(x1-x0) < tolerance) || (abs(f(x1)) < tolerance)
            break
        end
        x0 = x1
    end
    sequence
end

my_newton2 (generic function with 1 method)

In [6]:
s = my_newton2(x -> x^2 - 2.0, x -> 2*x, 1.0)

6-element Vector{Float64}:
 1.0
 1.5
 1.4166666666666667
 1.4142156862745099
 1.4142135623746899
 1.4142135623730951

In [7]:
abs.(s .- sqrt(2)) # error

6-element Vector{Float64}:
 0.41421356237309515
 0.08578643762690485
 0.002453104293571595
 2.1239014147411694e-6
 1.5947243525715749e-12
 0.0

In [8]:
-log10.( abs.(s .- sqrt(2)) ) # number of accurate digits

6-element Vector{Float64}:
  0.382775685337863
  1.066581366339708
  2.610283987399081
  5.672865645795637
 11.797314373736969
 Inf

In [9]:
# using BigFloats
s = my_newton2(x -> x^2 - 2.0, x -> 2*x, BigFloat(1.0), tolerance = 1e-50)

8-element Vector{BigFloat}:
 1.0
 1.5
 1.416666666666666666666666666666666666666666666666666666666666666666666666666661
 1.414215686274509803921568627450980392156862745098039215686274509803921568627451
 1.414213562374689910626295578890134910116559622115744044584905019200054371835385
 1.414213562373095048801689623502530243614981925776197428498289498623195824228933
 1.41421356237309504880168872420969807856967187537723400156101313311326525563035
 1.414213562373095048801688724209698078569671875376948073176679737990732478462102

In [10]:
@. -log10(abs(s - sqrt(BigFloat(2.0))))

8-element Vector{BigFloat}:
  0.3827756853378630783193995241920061132387223687070682985559555710056771255439819
  1.066581366339707351852537943108505253245634618876245138422338603138462440362357
  2.610283987399077141000103789472125815691398101943186141674542846582154033508534
  5.672865645792784578031610675292980325422044836658614274589161074076664786250617
 11.79727693731540182828937943111739390993500829878534781109792798559911206718055
 24.04609886812726521962096106811857676862166931759190182899734977637523982446757
 48.54374272975050223206253075446738688311550693137875471666123829095433756505891
 Inf

## Fixed point iteration

Section 4.2.

A fixed point $p$ of a function $g$ satisfies $p = g(p)$.

A fixed point iteration is a sequence $x_{k+1} = g(x_k)$ with some initial data $x_1$. 

For what functions and initial data does this series converge?

If $|g'(p)|<1$ and $x_1$ is close enough to $p$, the sequence will converge. If $|g'(p)| > 1$ the sequence will diverge.

The rate of convergence is characteried by the limit of the ratio of errors $\epsilon_k = x_k - p$,

$$\lim_{k\to\infty} \frac{\epsilon_{k+1}}{\epsilon_k} = \sigma.$$

If $\sigma<1$ the convergence is linear and the rate is $\sigma$. 

Numerically, you can't compute a limit and eventually limits on precision will cause errors in arithmetic to dominate the estimate of the limit. In practice we evaluate the limit by observing a few elements of the series.

There is a condition ([Lipshitz](https://mathworld.wolfram.com/LipschitzCondition.html), related to differentiability) which will guarantee convergence. This is one of many versions of the [fixed point theorem](https://mathworld.wolfram.com/FixedPointTheorem.html) ([more](https://en.wikipedia.org/wiki/Fixed-point_theorems)).



## Secant method

Our first use of the secant also required a bracket on the root (low and high estimates). This is not necessary and in fact, not really all that desirable. A better version replaces the derivative in Newton's method with a secant computed on the previous two estimates. You still need two points to start, but you don't attempt to keep a bracket on the root. See Function 4.4.4 from the text.

## Supralinear convergence

Section 4.4.

Superlinear convergence:
Suppose a sequence $x_k$ approaches limit $x^*$. If the error sequence $\epsilon_k=x_k - x^*$ satisfies

$$\lim_{k\to\infty} \frac{|\epsilon_{k+1}|}{|\epsilon_k|^\alpha} = L$$

for constants $\alpha >1$ and $L>0$, then the sequence has **superlinear convergence** with rate $\alpha$. 

Quadratic convergence is a particular case of superlinear convergence.

The text demonstrates that the rate of convergence for the secant method is the golden ratio $(1+ \sqrt{5})/2 \approx 1.618$.

## Multidimensional Newton

Section 4.5.

Solve a set of equations $f_i(x) = 0$.

Use a multidimensional Taylor expansion: $f(x+h) = f(x) + J(x)h + \dots$ where $J$ is the Jacobian (the matrix of first derivatives $\frac{\partial f_i}{\partial x_j}$ evaluated at $x$.

The single variable case translates almost exactly:

$$x_{k+1} = x_{k} - J^{-1}(x_k) f(x_k).$$

Naturally we will solve a linear system to get the increment to $x_k$ at each step. The sequence will be $x_{k+1} = x_k + s_k$ where $J(x_k) s_k = f(x_k)$.

## More great stuff

Not discussed in the class: Quasi-Newton and Non-linear least squares (sections 4.6, 4.7)

Some methods combine features of more than one of these methods: for example use interpolation (secant) or Newton with a starting value to find a bracket, then keep the bracket by carefully updating it.

The finite difference methods for the Jacobian and derivative (single variable) are often replaced with "automatic" differentiation; we will return to this in the next chapter.

One more root finding topic to come: interval arithmetic and root finding. This material is not in the textbook.